In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import HTML

In [50]:
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, precision_score, recall_score
import pandas
import numpy as np
import papermill as pm
import json
import matplotlib.pyplot as plt
import os
import uuid
from db import Result
import ast
import math
import scrapbook as sb
pandas.options.display.float_format = '{:,.3f}'.format
import pickle
from clinical_data_models import features_data

In [3]:
from evaluate import plot_learning_curve, plot_accuracy_curve, load, get_results, get_labels, transform_binary_probabilities, transform_binary_predictions, calculate_accuracy_loss, plot_confusion_matrix, plot_precision_recall, plot_roc_curve, calculate_pr_auc, calculate_confusion_matrix_stats, calculate_confusion_matrix, plot_precision_recall#, plot_tsne, plot_multiple_grad_cam
from data_gen import data
from run import characterize_data
from config import config

In [39]:
# T1 results
t1_results = Result.query.filter(Result.description == "t1c-t2-exp", Result.input_form == "t1c").all()
#t1_results = Result.query.filter(Result.description == "t1_final_200").all()

# T2 results
t2_results = Result.query.filter(Result.description == "t1c-t2-exp", Result.input_form == "t2").all()
#t2_results = Result.query.filter(Result.description == "t2_final_run_200").all()

# Clinical results
uuids = ["ea82b20c-ced8-4787-91f9-5eb3a15f5554",
"4af80ba2-96de-4ca4-a2e2-f003c419bb02",
"7b22fa31-5787-4fe1-a1a3-03f2385e2324",
"f617e4d1-068d-4c19-b864-bf4d61f11ab5",
"a1fd5687-622a-44c0-ae4f-6bdaab289b1e",
"228e571d-cd25-4bae-9983-7043d2294882",
"51d7476d-7445-4a85-9a52-38bf06d63b13",
"0c1a57f3-7183-42bf-92dc-28c278d150ac",
"4a7db6a9-f55e-4719-8f60-460e1dc7a697",
"6860ca52-0f15-4385-990d-0cb64fbfc18c",]
clinical_results = Result.query.filter(Result.uuid.in_(uuids)).all()
#t2_results = Result.query.filter(Result.description == "t2_final_run_200").all()

In [40]:
print(t1_results)
print(t2_results)
print(clinical_results)

[<Result accuracy: 0.5257731676101685>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5463917255401611>, <Result accuracy: 0.4845360815525055>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5360824465751648>, <Result accuracy: 0.47422680258750916>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5154638886451721>]
[<Result accuracy: 0.7010309100151062>, <Result accuracy: 0.6391752362251282>, <Result accuracy: 0.7010309100151062>, <Result accuracy: 0.6804123520851135>, <Result accuracy: 0.6804123520851135>, <Result accuracy: 0.6804123520851135>, <Result accuracy: 0.6494845151901245>, <Result accuracy: 0.6185566782951355>, <Result accuracy: 0.6391752362251282>, <Result accuracy: 0.6597937941551208>]
[<Result accuracy: 0.5876288659793815>, <Result accuracy: 0.5876288659793815>, <Result accuracy: 0.5876288659793815>, <Result accuracy: 0.5876288659793815>, <Result accuracy: 0.5876288659793815>, <Result accuracy: 0.5

In [6]:
train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t1", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t1 = {}
for result in t1_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    test_results = get_results(model, test)
    test_labels = get_labels(test)
    stats = calculate_confusion_matrix_stats(labels, results)
    #all_results_t1[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_t1[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}

train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t2", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t2 = {}
for result in t2_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    test_results = get_results(model, test)
    test_labels = get_labels(test)
    stats = calculate_confusion_matrix_stats(labels, results)
    #all_results_t2[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_t2[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}


  0%|          | 0/678 [00:00<?, ?it/s]

                outcome_pos  outcome_neg  outcome_3    age  sex  \
patientID                                                         
bone-brown-1              0            0          0 52.000    1   
bone-brown-10             0            0          0 52.000    0   
bone-brown-100            1            1          2 22.000    1   
bone-brown-101            0            0          0 88.000    0   
bone-brown-102            0            0          0 56.000    1   
...                     ...          ...        ...    ...  ...   
bone-penn-650             1            1          2 20.000    1   
bone-penn-651             0            0          0 16.000    1   
bone-penn-652             1            1          2  8.000    0   
bone-penn-653             1            1          2 16.000    1   
bone-penn-654             1            1          2  3.000    1   

                                                         location        sort  
patientID                                       

  0%|          | 1/678 [00:00<06:25,  1.76it/s]

bone-brown-100
I'm doing the thing.


  0%|          | 2/678 [00:00<04:48,  2.34it/s]

...

 98%|█████████▊| 95/97 [00:11<00:00,  8.34it/s]


################################################################################
Exception occurred for: outcome_pos                                                    1
outcome_neg                                                    1
outcome_3                                                      2
age                                                         17.0
sex                                                            1
location       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
sort                                                    external
Name: bone-china-359, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-china-359-t1c-imagingVolume.nrrd'
T2 image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 448, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in load_image
    image, _ = nrrd.read(

100%|██████████| 97/97 [00:11<00:00,  8.42it/s]



################################################################################
Exception occurred for: outcome_pos                                                    1
outcome_neg                                                    1
outcome_3                                                      2
age                                                         38.0
sex                                                            1
location       [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
sort                                                    external
Name: bone-china-361, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-china-361-t1c-imagingVolume.nrrd'
T2 image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 448, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in load_image
    image, _ = nrrd.read(

/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/home/ubuntu/minico

In [47]:
train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="features", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)

  0%|          | 0/678 [00:00<?, ?it/s]

                outcome_pos  outcome_neg  outcome_3    age  sex  \
patientID                                                         
bone-brown-1              0            0          0 52.000    1   
bone-brown-10             0            0          0 52.000    0   
bone-brown-100            1            1          2 22.000    1   
bone-brown-101            0            0          0 88.000    0   
bone-brown-102            0            0          0 56.000    1   
...                     ...          ...        ...    ...  ...   
bone-penn-650             1            1          2 20.000    1   
bone-penn-651             0            0          0 16.000    1   
bone-penn-652             1            1          2  8.000    0   
bone-penn-653             1            1          2 16.000    1   
bone-penn-654             1            1          2  3.000    1   

                                                         location        sort  
patientID                                       

  0%|          | 1/678 [00:00<04:33,  2.48it/s]

bone-brown-100
I'm doing the thing.

################################################################################
Exception occurred for: outcome_pos                                                    1
outcome_neg                                                    1
outcome_3                                                      2
age                                                         22.0
sex                                                            1
location       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
sort                                                       train
Name: bone-brown-100, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-brown-100-t1c-imagingVolume.nrrd'
T1C image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 480, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in

  1%|          | 5/678 [00:02<04:32,  2.47it/s]

...

I'm doing the thing.

################################################################################
Exception occurred for: outcome_pos                                                    0
outcome_neg                                                    0
outcome_3                                                      0
age                                                         20.0
sex                                                            1
location       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
sort                                                    external
Name: bone-china-344, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-china-344-t1c-imagingVolume.nrrd'
T1C image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 480, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in load_image
   

100%|██████████| 97/97 [00:01<00:00, 60.00it/s]

bone-china-354
I'm doing the thing.

################################################################################
Exception occurred for: outcome_pos                                                    0
outcome_neg                                                    0
outcome_3                                                      0
age                                                         30.0
sex                                                            0
location       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
sort                                                    external
Name: bone-china-354, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-china-354-t1c-imagingVolume.nrrd'
T1C image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 480, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in


/home/ubuntu/Bone-MRI/data_gen.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.features = np.array(features)


AttributeError: 'LogisticRegression' object has no attribute 'predict_generator'

In [54]:
train_set, train_labels, val_set, val_labels, test_set, test_labels, external_set, external_labels = features_data(train, validation, test, external)
all_results_clinical = {}
for result in clinical_results:
    MODEL = "{}/models/{}_features.sav".format(config.OUTPUT, result.uuid)
    model = pickle.load(open(MODEL, 'rb'))
    results = model.predict(val_set).tolist()
    probabilities=model.predict_proba(val_set).tolist()
    labels = val_labels
    test_results = model.predict(test_set).tolist()
    test_labels = test_labels
    #stats = calculate_confusion_matrix_stats(labels, probabilities)
    #all_results_clinical[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_clinical[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}


In [55]:
import pandas as pd
print("T1")
print(pd.DataFrame.from_dict(all_results_t1, orient='index'))
print("T2")
print(pd.DataFrame.from_dict(all_results_t2, orient='index'))
print("Clinical")
print(pd.DataFrame.from_dict(all_results_clinical, orient='index'))

T1
                                      roc_auc_val  roc_auc_test
ac00a260-affc-4846-bb4c-a591e4332728        0.502         0.519
6bb799dc-f454-423a-8dbc-37d18159b2df        0.436         0.488
26e0bfee-eadc-4da0-ad8e-e8124a2ebd52        0.479         0.519
c26d1754-30e7-482d-af0c-93834c484085        0.617         0.560
8ac52b0c-9cee-4664-8b34-931d211fabf8        0.568         0.481
2e2066f9-5519-423e-93fe-13d367c8047e        0.521         0.528
5fb2d484-59a3-4524-aa56-3b4217138930        0.609         0.527
59744f0d-0b07-4d35-ab23-230d20469780        0.629         0.533
8d1cf33c-ec4d-44ce-847d-4830671ab803        0.503         0.483
b7250c9d-76e8-404f-bac6-34604b60ffa3        0.725         0.462
T2
                                      roc_auc_val  roc_auc_test
8af9ab00-4d86-41ce-8a48-659af724d622        0.746         0.714
2d6f781e-a7b9-48c9-a9cd-a55d8240315d        0.735         0.594
92f4101e-9c5b-48c0-ad68-9530d345a968        0.745         0.656
44968cdd-c2ab-4eac-833a-19ef08ad47

In [56]:
import math
import statistics
def adjusted_wald(p, n, z=1.96):
    p_adj = (n * p + (z**2)/2)/(n+z**2)
    n_adj = n + z**2
    span = z * math.sqrt(p_adj*(1-p_adj)/n_adj)
    return max(0, p_adj - span), min(p_adj + span, 1.0)

avg_t1_val = statistics.mean([all_results_t1[data]["roc_auc_val"] for data in all_results_t1])
avg_t1_test = statistics.mean([all_results_t1[data]["roc_auc_test"] for data in all_results_t1])
ci_val = adjusted_wald(avg_t1_val, 10)
ci_test = adjusted_wald(avg_t1_test, 10)
print("T1C val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t1_val, *ci_val))
print("T1C test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t1_test, *ci_test))

avg_t2_val = statistics.mean([all_results_t2[data]["roc_auc_val"] for data in all_results_t2])
avg_t2_test = statistics.mean([all_results_t2[data]["roc_auc_test"] for data in all_results_t2])
ci_val = adjusted_wald(avg_t2_val, 10)
ci_test = adjusted_wald(avg_t2_test, 10)
print("T2 val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t2_val, *ci_val))
print("T2 test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t2_test, *ci_test))

avg_clinical_val = statistics.mean([all_results_clinical[data]["roc_auc_val"] for data in all_results_clinical])
avg_clinical_test = statistics.mean([all_results_clinical[data]["roc_auc_test"] for data in all_results_clinical])
ci_val = adjusted_wald(avg_clinical_val, 10)
ci_test = adjusted_wald(avg_clinical_test, 10)
print("Clinical val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_clinical_val, *ci_val))
print("Clinical test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_clinical_test, *ci_test))

T1 val
0.56 (0.28-0.81)
T1 test
0.51 (0.24-0.77)
T2 val
0.72 (0.41-0.91)
T2 test
0.66 (0.36-0.87)
Clinical val
0.59 (0.30-0.83)
Clinical test
0.67 (0.36-0.88)
